In [1]:
import numpy as np
import pandas as pd

In [2]:
titanic_df = pd.read_csv('./Titanic_Data-master/train.csv')
test_df = pd.read_csv('./Titanic_Data-master/test.csv')
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


#### 数据集中的字段描述：

<p>PassengerId:乘客编号
<p>Survived:是否幸存
<p>Pclass:船票等级
<p>Name:乘客姓名
<p>Sex:乘客性别
<p>SibSp:亲戚数量（兄妹、配偶数）
<p>Parch:亲戚数量（父母、子女数）
<p>Ticket:船票号码
<p>Fare:船票价格
<p>Cabin:船舱
<p>Embarked:登录港口

In [3]:
# 数据探索
print(titanic_df.info())
print('-'*30)
print(titanic_df.describe())
print('-'*30)
print(titanic_df.describe(include=['O']))
print('-'*30)
print(titanic_df.head())
#print('-'*30)
#print(titanic_df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
------------------------------
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.00000

In [10]:
# Age缺失值处理
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
test_df['Age'].fillna(test_df['Age'].mean(), inplace=True)

# Fare缺失值处理
test_df['Fare'].fillna(test_df['Fare'].mean(), inplace=True)

# Embarked缺失值处理
#print(titanic_df.Embarked.value_counts())
titanic_df.Embarked.fillna('S', inplace=True)
#print(titanic_df.Embarked.value_counts())
test_df.Embarked.fillna('S', inplace=True)

In [24]:
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_train = titanic_df[features]
y_train = titanic_df['Survived']
X_test = test_df[features]

In [25]:
# 离散型数据转换为数值型
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse=False)
X_train = dvec.fit_transform(X_train.to_dict(orient='record'))
X_test = dvec.fit_transform(X_test.to_dict(orient='record'))

print(dvec.feature_names_)

#from scipy import stats
#print(stats.describe(X_test))

['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']
DescribeResult(nobs=418, minmax=(array([0.17, 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ]), array([ 76.    ,   1.    ,   1.    ,   1.    , 512.3292,   9.    ,
         3.    ,   1.    ,   1.    ,   8.    ])), mean=array([30.27259036,  0.24401914,  0.11004785,  0.64593301, 35.62718849,
        0.3923445 ,  2.26555024,  0.36363636,  0.63636364,  0.44736842]), variance=array([1.59631454e+02, 1.84916182e-01, 9.81721800e-02, 2.29252005e-01,
       3.11816149e+03, 9.63202644e-01, 7.08690464e-01, 2.31959887e-01,
       2.31959887e-01, 8.04177710e-01]), skewness=array([ 0.51086883,  1.19198323,  2.49211004, -0.61030523,  3.67833924,
        4.63774241, -0.53225156,  0.56694671, -0.56694671,  4.15336349]), kurtosis=array([ 0.8583274 , -0.57917597,  4.21061244, -1.62752753, 17.74269785,
       31.02370168, -1.38050242, -1.67857143, -1.67857143, 26.16848189]))


In [29]:
# 建模
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy')

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# 准确率
acc_decission_tree = clf.score(X_train, y_train)
print(u'score 准确率为：%.4lf' % acc_decission_tree)

# K-Fold CV
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y_train, cv=10)
print(u'cross_val_score 准确率为：%.4lf' % np.mean(scores))

score 准确率为：0.9820
cross_val_score 准确率为：0.7746


In [32]:
# Graphviz
import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("titanic_tree")
graph.view('titanic_graph')

'titanic_graph.pdf'